In [33]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)

# Informações desbalanceadas analisadas no arquivo anterior
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

base.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


In [3]:
base = base[base.price > 10]
base = base.loc[base.price <350000]

In [4]:
valores = {'vehicleType': 'limousine', 'gearbox' : 'manuell', 'model' : 'golf',
              'fuelType' : 'benzin', 'notRepairedDamage' : 'nein'}

In [5]:
base = base.fillna(value = valores)

In [6]:
previsores = base.iloc[:, 1:13].values # Todas as colunas, exceto a 'price'
preco_real = base.iloc[:, 0].values # Coluna 'price'

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder_previsores = LabelEncoder()

previsores[:,0] = label_encoder_previsores.fit_transform(previsores[:,0])
previsores[:,1] = label_encoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = label_encoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = label_encoder_previsores.fit_transform(previsores[:,5])
previsores[:,8] = label_encoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = label_encoder_previsores.fit_transform(previsores[:,9])
previsores[:,10] = label_encoder_previsores.fit_transform(previsores[:,10])

In [24]:
previsores.shape

(359291, 323)

In [32]:
onehotencoder = OneHotEncoder(categorical_features = [0, 1, 3, 5, 8, 9, 10] )
previsores = onehotencoder.fit_transform(previsores).toarray()

/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [25]:
def criar_rede():
    regressor = Sequential()

    # Mesma regra, (entrada + saida) / 2, nesse caso (323 + 1) / 2 = 162.
    regressor.add(Dense(units = 162, activation = 'relu', input_dim = 323 ))
    regressor.add(Dense(units = 162, activation = 'relu'))
    regressor.add(Dense(units = 1, activation = 'linear'))

    regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['mean_absolute_error'])

    regressor.fit(previsores, preco_real, batch_size = 300, epochs = 100)
    
    return regressor

In [26]:
regressor = KerasRegressor(build_fn = criar_rede,
                              epochs = 100,
                               batch_size = 300)

In [28]:
resultados = cross_val_score(estimator = regressor,
                                X = previsores, y = preco_real,
                                cv = 10, scoring = 'neg_mean_absolute_error')

W0831 01:58:54.354819 140415376901952 deprecation_wrapper.py:119] From /home/jarbasjr/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0831 01:58:54.451398 140415376901952 deprecation_wrapper.py:119] From /home/jarbasjr/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/100
359291/359291 [==============================] - 6s 15us/step - loss: 3825.7374 - mean_absolute_error: 3825.7374
Epoch 2/100
359291/359291 [==============================] - 4s 12us/step - loss: 3283.4640 - mean_absolute_error: 3283.4640
Epoch 3/100
359291/359291 [==============================] - 5s 15us/step - loss: 3006.7231 - mean_absolute_error: 3006.7231
Epoch 4/100
359291/359291 [==============================] - 5s 14us/step - loss: 2863.4305 - mean_absolute_error: 2863.4305
Epoch 5/100
359291/359291 [==============================] - 5s 13us/step - loss: 2808.9136 - mean_absolute_error: 2808.9136
Epoch 6/100
359291/359291 [==============================] - 5s 13us/step - loss: 2761.2134 - mean_absolute_error: 2761.2134
Epoch 7/100
359291/359291 [==============================] - 5s 13us/step - loss: 2715.8383 - mean_absolute_error: 2715.8383
Epoch 8/100
359291/359291 [==============================] - 5s 13us/step - loss: 2674.2516 - mean_absolute_error: 2674.2516


359291/359291 [==============================] - 5s 13us/step - loss: 2289.6909 - mean_absolute_error: 2289.6909
Epoch 67/100
359291/359291 [==============================] - 5s 14us/step - loss: 2293.4416 - mean_absolute_error: 2293.4416
Epoch 68/100
359291/359291 [==============================] - 5s 13us/step - loss: 2279.7177 - mean_absolute_error: 2279.7177
Epoch 69/100
359291/359291 [==============================] - 5s 15us/step - loss: 2290.5629 - mean_absolute_error: 2290.5629
Epoch 70/100
359291/359291 [==============================] - 6s 16us/step - loss: 2281.3669 - mean_absolute_error: 2281.3669
Epoch 71/100
359291/359291 [==============================] - 5s 13us/step - loss: 2275.5356 - mean_absolute_error: 2275.5356
Epoch 72/100
359291/359291 [==============================] - 5s 13us/step - loss: 2275.0725 - mean_absolute_error: 2275.0725
Epoch 73/100
359291/359291 [==============================] - 5s 13us/step - loss: 2269.3377 - mean_absolute_error: 2269.3377
Epoch

323361/323361 [==============================] - 4s 13us/step - loss: 2209.3365 - mean_absolute_error: 2209.3365
Epoch 32/100
323361/323361 [==============================] - 4s 13us/step - loss: 2220.4574 - mean_absolute_error: 2220.4574
Epoch 33/100
323361/323361 [==============================] - 4s 13us/step - loss: 2214.4979 - mean_absolute_error: 2214.4979
Epoch 34/100
323361/323361 [==============================] - 4s 13us/step - loss: 2219.9808 - mean_absolute_error: 2219.9808
Epoch 35/100
323361/323361 [==============================] - 4s 13us/step - loss: 2202.0159 - mean_absolute_error: 2202.0159
Epoch 36/100
323361/323361 [==============================] - 4s 13us/step - loss: 2213.2896 - mean_absolute_error: 2213.2896
Epoch 37/100
323361/323361 [==============================] - 4s 13us/step - loss: 2217.5042 - mean_absolute_error: 2217.5042
Epoch 38/100
323361/323361 [==============================] - 4s 13us/step - loss: 2212.9930 - mean_absolute_error: 2212.9930
Epoch

323361/323361 [==============================] - 3s 10us/step - loss: 2177.6307 - mean_absolute_error: 2177.6307
Epoch 97/100
323361/323361 [==============================] - 3s 10us/step - loss: 2182.0606 - mean_absolute_error: 2182.0606
Epoch 98/100
323361/323361 [==============================] - 3s 10us/step - loss: 2171.6167 - mean_absolute_error: 2171.6167
Epoch 99/100
323361/323361 [==============================] - 3s 10us/step - loss: 2192.4971 - mean_absolute_error: 2192.4971
Epoch 100/100
323361/323361 [==============================] - 3s 10us/step - loss: 2177.9998 - mean_absolute_error: 2177.9998
Epoch 1/100
359291/359291 [==============================] - 4s 11us/step - loss: 3814.7339 - mean_absolute_error: 3814.7339
Epoch 2/100
359291/359291 [==============================] - 4s 10us/step - loss: 3386.2458 - mean_absolute_error: 3386.2458
Epoch 3/100
359291/359291 [==============================] - 4s 10us/step - loss: 3122.6942 - mean_absolute_error: 3122.6942
Epoch 4

...

359291/359291 [==============================] - 4s 10us/step - loss: 2248.2776 - mean_absolute_error: 2248.2776
Epoch 90/100
359291/359291 [==============================] - 4s 10us/step - loss: 2244.9066 - mean_absolute_error: 2244.9066
Epoch 91/100
359291/359291 [==============================] - 4s 10us/step - loss: 2241.6065 - mean_absolute_error: 2241.6065
Epoch 92/100
359291/359291 [==============================] - 4s 10us/step - loss: 2253.9360 - mean_absolute_error: 2253.9360
Epoch 93/100
359291/359291 [==============================] - 4s 10us/step - loss: 2251.3434 - mean_absolute_error: 2251.3434
Epoch 94/100
359291/359291 [==============================] - 5s 13us/step - loss: 2235.5606 - mean_absolute_error: 2235.5606
Epoch 95/100
359291/359291 [==============================] - 5s 13us/step - loss: 2238.3355 - mean_absolute_error: 2238.3355
Epoch 96/100
359291/359291 [==============================] - 5s 13us/step - loss: 2235.6439 - mean_absolute_error: 2235.6439
Epoch

323362/323362 [==============================] - 4s 11us/step - loss: 2205.5530 - mean_absolute_error: 2205.5530
Epoch 55/100
323362/323362 [==============================] - 4s 13us/step - loss: 2200.4664 - mean_absolute_error: 2200.4664
Epoch 56/100
323362/323362 [==============================] - 4s 11us/step - loss: 2210.8816 - mean_absolute_error: 2210.8816
Epoch 57/100
323362/323362 [==============================] - 3s 11us/step - loss: 2200.7649 - mean_absolute_error: 2200.7649
Epoch 58/100
323362/323362 [==============================] - 3s 11us/step - loss: 2206.1468 - mean_absolute_error: 2206.1468
Epoch 59/100
323362/323362 [==============================] - 3s 11us/step - loss: 2197.7018 - mean_absolute_error: 2197.7018
Epoch 60/100
323362/323362 [==============================] - 3s 11us/step - loss: 2197.9852 - mean_absolute_error: 2197.9852
Epoch 61/100
323362/323362 [==============================] - 4s 12us/step - loss: 2207.2266 - mean_absolute_error: 2207.2266
Epoch

359291/359291 [==============================] - 7s 21us/step - loss: 2409.9927 - mean_absolute_error: 2409.9927
Epoch 20/100
359291/359291 [==============================] - 8s 22us/step - loss: 2424.7113 - mean_absolute_error: 2424.7113
Epoch 21/100
359291/359291 [==============================] - 8s 21us/step - loss: 2408.1027 - mean_absolute_error: 2408.1027
Epoch 22/100
359291/359291 [==============================] - 7s 21us/step - loss: 2411.2796 - mean_absolute_error: 2411.2796
Epoch 23/100
359291/359291 [==============================] - 8s 21us/step - loss: 2391.4413 - mean_absolute_error: 2391.4413
Epoch 24/100
359291/359291 [==============================] - 6s 17us/step - loss: 2394.8163 - mean_absolute_error: 2394.8163
Epoch 25/100
359291/359291 [==============================] - 5s 13us/step - loss: 2383.9344 - mean_absolute_error: 2383.9344
Epoch 26/100
359291/359291 [==============================] - 5s 15us/step - loss: 2385.2531 - mean_absolute_error: 2385.2531
Epoch

359291/359291 [==============================] - 4s 10us/step - loss: 2261.0746 - mean_absolute_error: 2261.0746
Epoch 85/100
359291/359291 [==============================] - 4s 10us/step - loss: 2250.2845 - mean_absolute_error: 2250.2845
Epoch 86/100
359291/359291 [==============================] - 4s 12us/step - loss: 2262.9967 - mean_absolute_error: 2262.9967
Epoch 87/100
359291/359291 [==============================] - 4s 12us/step - loss: 2245.9447 - mean_absolute_error: 2245.9447
Epoch 88/100
359291/359291 [==============================] - 4s 12us/step - loss: 2244.9412 - mean_absolute_error: 2244.9412
Epoch 89/100
359291/359291 [==============================] - 4s 12us/step - loss: 2245.6700 - mean_absolute_error: 2245.6700
Epoch 90/100
359291/359291 [==============================] - 4s 11us/step - loss: 2251.3630 - mean_absolute_error: 2251.3630
Epoch 91/100
359291/359291 [==============================] - 4s 11us/step - loss: 2247.8577 - mean_absolute_error: 2247.8577
Epoch

323362/323362 [==============================] - 4s 12us/step - loss: 2199.4446 - mean_absolute_error: 2199.4446
Epoch 50/100
323362/323362 [==============================] - 3s 10us/step - loss: 2217.3661 - mean_absolute_error: 2217.3661
Epoch 51/100
323362/323362 [==============================] - 3s 11us/step - loss: 2200.8127 - mean_absolute_error: 2200.8127
Epoch 52/100
323362/323362 [==============================] - 4s 11us/step - loss: 2196.8033 - mean_absolute_error: 2196.8033
Epoch 53/100
323362/323362 [==============================] - 4s 11us/step - loss: 2203.3833 - mean_absolute_error: 2203.3833
Epoch 54/100
323362/323362 [==============================] - 4s 12us/step - loss: 2203.0643 - mean_absolute_error: 2203.0643
Epoch 55/100
323362/323362 [==============================] - 4s 11us/step - loss: 2215.5736 - mean_absolute_error: 2215.5736
Epoch 56/100
323362/323362 [==============================] - 4s 11us/step - loss: 2195.9368 - mean_absolute_error: 2195.9368
Epoch

In [31]:
media = resultados.mean()
media

-2162.3548873082364

In [ ]:
desvio_padrao = resultados.std()